In [ ]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

📌 **Комментарий**: Подключение библиотек Selenium для работы с веб-страницами. 
- `WebDriverWait` и `expected_conditions` используются для ожидания загрузки элементов перед их обработкой.


In [ ]:
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
import time
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys 

# Инициализация браузера Chrome с настройками обхода защиты Cloudflare
options = uc.ChromeOptions()
options.add_argument('--no-sandbox')
options.add_argument('--disable-gpu')
options.add_argument('--disable-dev-shm-usage')

driver = uc.Chrome(options=options)

# Задержка для загрузки страницы
time.sleep(2)

# URL страницы ATC DrugBank
url = "https://go.drugbank.com/atc?query=A10B"
driver.get(url)
time.sleep(3)  # Ожидание для обработки Cloudflare-защиты

# Найти поле поиска и ввести запрос
xpath = '/html/body/main/div/div[3]/div[1]/form/div/input'
fld = driver.find_element(By.XPATH, xpath)
fld.click()
drug_name = 'A10B'  # Код группы лекарств от диабета
fld.send_keys(drug_name)
fld.send_keys(Keys.RETURN)

📌 **Комментарий**: Инициализация браузера Chrome с обходом защиты Cloudflare и загрузка страницы. 
- `undetected_chromedriver` помогает обходить автоматические проверки Cloudflare.
- `driver.get(url)`: загружает веб-страницу ATC-классификации.
- `fld.send_keys(drug_name)`: вводит поисковый запрос в поле.
- `Keys.RETURN`: имитирует нажатие Enter.


In [ ]:
# Ожидание появления дерева категорий
wait = WebDriverWait(driver, 10)
wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, ".jstree")))

# Разворачивание узлов дерева
nodes = [1,2]
while len(nodes) > 0:
    nodes = driver.find_elements(By.CSS_SELECTOR, ".jstree-node:not(.jstree-open)")
    for node in nodes:
        try:
            expander = node.find_element(By.CSS_SELECTOR, ".jstree-ocl")
            expander.click()
            time.sleep(0.5)  # Ожидание анимации
        except:
            continue

# Ожидание, пока все узлы будут развернуты
wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".jstree-open")))

📌 **Комментарий**: Разворачивание узлов дерева ATC-классификации. 
- Цикл находит все **свернутые** (`not(.jstree-open)`) элементы дерева и **раскрывает** их.
- Используется `expander.click()` для клика по элементам.
- `time.sleep(0.5)`: небольшая задержка для корректной работы анимации.


In [ ]:
page1 = driver.page_source
with open('page_expanded.html', 'w') as f:
    f.write(page1)

📌 **Комментарий**: Сохранение HTML-кода страницы после раскрытия ATC-дерева.
- `driver.page_source` получает HTML-код текущей страницы.
- Файл `page_expanded.html` используется для дальнейшего анализа структуры страницы.


In [ ]:
with open('page_expanded.html', 'r') as f:
    page1 = f.readlines()

In [ ]:
page1

In [ ]:
from bs4 import BeautifulSoup

# Преобразуем HTML-код страницы в объект BeautifulSoup
soup1 = BeautifulSoup(''.join(page1))  # Используется библиотека XML-парсинга
print(type(soup1))  # Выводим тип объекта

📌 **Комментарий**: Использование BeautifulSoup для парсинга HTML-страницы. 
- `soup1 = BeautifulSoup(page1, 'html.parser')` преобразует HTML в удобный для анализа объект.
- Вывод типа `soup1` подтверждает успешную обработку.


In [ ]:
# soup = BeautifulSoup(page1, "html.parser")

for child in soup.descendants:

    if child.name:
        print(child.name)

📌 **Комментарий**
- Цикл проходит по всем элементам (descendants) HTML-страницы.
- child.name выводит названия всех тегов (div, ul, li и т. д.).
- Это помогает понять структуру HTML-документа перед извлечением данных.

Это помогает понять структуру HTML-документа перед извлечением данных.

In [ ]:
res = soup1.find_all("ul", recursive=True)
# [x.text for x in res]  # Альтернативный способ вывести текст
# res

📌 **Комментарий**: Извлечение всех списков `<ul>` из структуры ATC-классификации. 
- `find_all('ul')` ищет все элементы списка, где хранятся подкатегории ATC.
- Это помогает понять структуру дерева ATC.


In [ ]:
res = soup1.find("div", attrs={'id' : 'category-jstree'})
res

📌 **Комментарий**: Извлечение главного контейнера с категорией ATC. 
- Ищет div-элемент с id="category-jstree", который, вероятно, содержит дерево ATC-классификации.
- Если этот div найден, то он будет использоваться для дальнейшего анализа.


In [ ]:
res = soup1.find("i", attrs={'role' : 'presentation'})
res.text

📌 Комментарий:

- find("i", attrs={'role': 'presentation'}) ищет первый элемент <i> с атрибутом role="presentation".
- .text извлекает текстовое содержимое этого элемента.
- Этот тег может использоваться для представления иконок или декоративных элементов в HTML-странице.

In [ ]:
res = soup1.find("div", attrs={'id' : 'category-jstree'})
res

📌 Комментарий:

- Аналогично ячейке 18, выполняется поиск div с id="category-jstree".
- Возможно, повторяется для дополнительной проверки или отладки.

In [ ]:
soup1.text

📌 Комментарий:

- .text извлекает весь текст со страницы, удаляя HTML-теги.
Может использоваться для быстрого анализа содержимого, но без структуры.


In [ ]:
res = soup1.findAll("a")# attrs={'id' : 'jstree-anchor'})
res

📌 **Комментарий**: Извлечение всех ссылок `<a>` на странице.
- `findAll('a')` выбирает все гиперссылки, содержащие ATC-коды и их описания.


In [ ]:
texts = np.array([x.text for x in res])

📌 Комментарий:

- np.array([...]) создает массив NumPy из текстового содержимого всех найденных ссылок <a>.
- Это позволяет в дальнейшем анализировать полученные категории ATC в структурированном формате.

In [ ]:
import numpy as np

# Выбираем только те элементы, у которых есть хотя бы две скобки '()'
idx = np.nonzero([(len((set(x)) & set('()')) >= 2) for x in texts])[0]

# Получаем отфильтрованные элементы
entries = texts[idx]

📌 **Комментарий**: Фильтрация элементов, содержащих коды ATC.
- Выбираем только те элементы, у которых есть **две скобки** `()`.
- Используется `np.nonzero([...])[0]` для получения индексов таких элементов.


In [ ]:
entries[1]

📌 Комментарий:

- Выводит второй (индекс 1) элемент из массива entries, содержащего отфильтрованные ATC-коды.
- Используется для проверки корректности данных.

In [ ]:
categories = dict()
unknowns = 0

for i, entry in enumerate(entries):
    key = entry[entry.rfind('('):]  # Извлекаем часть с кодом ATC в скобках
    atc = entry[:entry.rfind('(')]  # Извлекаем текстовое описание ATC-кода
    categories[key[1:-1]] = atc[:-1]  # Заполняем словарь

📌 **Комментарий**: Создается словарь categories, где:
- Ключ — код ATC (например, "A10B").
- Значение — описание ATC-классификации.
- entry.rfind('(') находит последнюю открывающую скобку, чтобы разделить код ATC и его описание.



In [ ]:
categories

In [ ]:
import pandas as pd

# Преобразуем словарь в DataFrame
df = pd.DataFrame(data=categories.values(), index=categories.keys())
df.reset_index(inplace=True)
df.columns = ['ATC', 'ATC_info']

df.head()  # Вывод первых строк

- Преобразует словарь categories в таблицу (DataFrame).
- reset_index(inplace=True) сбрасывает индекс, делая код ATC отдельным столбцом.
- df.columns = ['ATC', 'ATC_info'] задаёт названия столбцов:
- ATC — код ATC.
- ATC_info — описание классификации.
- df.head() выводит первые строки таблицы.

📌 **Комментарий**: Преобразование словаря ATC в DataFrame.
- `pd.DataFrame(data=categories.values(), index=categories.keys())` создаёт таблицу из словаря.
- `reset_index(inplace=True)`: сбрасывает индексы, делая ATC-код отдельным столбцом.


In [ ]:
def reconstruct_level(df, atc):
    ATC_chain = []
    while len(atc) > 0:
        if atc in df['ATC'].values:
            ATC_chain.append(atc)
        atc = atc[:-1]  # Убираем последний символ, чтобы подняться на уровень выше
    return ATC_chain

📌 Комментарий:

- Функция reconstruct_level(df, atc) восстанавливает иерархию ATC-классификации.
Принцип работы:
- Добавляет atc в ATC_chain, если код есть в df['ATC'].
- Обрезает последний символ atc = atc[:-1], чтобы подняться на уровень выше в иерархии ATC-классификации.
- Повторяет этот процесс, пока atc не станет пустым.

In [ ]:
print(reconstruct_level(df, 'A10BJ06'))
levels = []
for i in range(df.shape[0]):
    levels.append(reconstruct_level(df, df.iloc[i,:]['ATC']))

📌 **Комментарий**: Применение функции `reconstruct_level()` ко всем ATC-кодам.
- print(reconstruct_level(df, 'A10BJ06')) демонстрирует восстановленную иерархию для A10BJ06.
- Цикл обрабатывает весь DataFrame, применяя reconstruct_level() к каждому коду ATC.


In [ ]:
df['ATC_hierarchy'] = levels

📌 **Комментарий**: Добавление столбца `ATC_hierarchy` в DataFrame.
- Теперь у каждого ATC-кода есть соответствующая иерархическая цепочка.


In [ ]:
df[df['ATC'] == 'A10']

📌 **Комментарий**: Фильтрация таблицы, оставляя только категорию `A10` (препараты от диабета).


In [ ]:
df['used_in_diabetes'] = df['ATC'].apply(lambda x: 'A10' in x)

📌 **Комментарий**: Определение, связан ли препарат с лечением диабета.
- `df['used_in_diabetes'] = df['ATC'].apply(lambda x: 'A10' in x)`: добавляет столбец.
- Значение `True`, если код ATC начинается с `A10` (категория диабета).


In [ ]:
df['number_of_children'] = df['ATC'].apply(lambda x: np.sum([x in y for y in df['ATC'].values]))

In [ ]:
df.head()

In [ ]:
df.to_csv('data/ATC_categories_full.csv')

In [ ]:
df_diabetis = df[(df['number_of_children'] == 1) & (df['used_in_diabetes'])]

In [ ]:
df_diabetis.to_csv('data/ATC_categories_diabetis.csv')

In [ ]:
df_diabetis['ATC_info'].value_counts()

In [ ]:
df_diabetis[df_diabetis['ATC_info'] == 'Insulin (human)']

In [ ]:
# df_diabetis[df['ATC_info'] == 'Combinations']

In [ ]:
df_diabetis[df_diabetis['ATC_info'] == 'Dapagliflozin']

In [ ]:
df[df['ATC_info'] == 'Semaglutide']

In [ ]:
df[df['ATC'] == 'A10BJ06']

In [ ]:
df[df['ATC'] == 'A10BJ']

In [ ]:
df[df['ATC'] == 'A10B']

In [ ]:
df[df['ATC'] == 'A10']

In [ ]:
df[df['ATC'] == 'A']

In [ ]:
df_diabetis

In [ ]:
df_diabetis[df_diabetis['ATC_info'].apply(lambda x: ' and ' in x)]

In [ ]:
df_diabetis_solo_drugs = df_diabetis[~df_diabetis['ATC_info'].apply(lambda x: ' and ' in x)]

In [ ]:
df_diabetis_solo_drugs['ATC'].values

In [ ]:
import shutil
for atc in df_diabetis_solo_drugs['ATC'].values:
    shutil.copy2(f'drug_pages/{atc}.html', f'drug_pages_solo/{atc}.html')